In [1]:
# To handle relative importing
import os
import sys
module_paths = [
    os.path.abspath(os.path.join('..')),
]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2
# import necessary files
from tqdm import tqdm
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
# dataloader
from src.dataloaders.dataloader import ImgNetDataset
from torch.utils.data import DataLoader

In [3]:
# imgnet 1k
dataset = ImgNetDataset("/root/CS570-Final-Project/datasets/imgnet1k_original/val.json", type='val')
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [7]:
from PIL import Image

# resnet50 torch

import time
from torchvision import transforms

resnet50 = torchvision.models.resnet50()
load_from_checkpoint = '/root/CS570-Final-Project/pt_models/rn50_relabel_78.9.pth'
state_dict = torch.load(load_from_checkpoint)
items = list(state_dict.items())
for k, v in items:
    if k.startswith('module.'):
        state_dict[k[len('module.'):]] = state_dict.pop(k)
resnet50.load_state_dict(state_dict)
resnet50.eval()
resnet50.cuda()

json_path = '/root/CS570-Final-Project/datasets/imgnet1k_original/val.json'
with open(json_path, 'r') as f:
    json_data = json.load(f)
    json_data = json_data['data']
    # random.shuffle(json_data)

acc, cnt = 0, 0
ms = 1000
B = 256
img_L, labels_L = [], []
for idx, item in enumerate(tqdm(json_data)):
    start_time = time.time()
    img_path = item['img_path']
    img = Image.open(img_path).convert("RGB")
    img = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(256, antialias=True),
                transforms.CenterCrop(224),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])(img).unsqueeze(0).cuda()
    end_time = time.time()

    labels = torch.tensor(item['labels'])
    y = torch.zeros(1000)
    if len(labels) > 0:
        y[labels] = 1

    y = y.unsqueeze(0).cuda()

    img_L.append(img)
    labels_L.append(y)

    if len(img_L) == B or idx == len(json_data) - 1:
        img = torch.cat(img_L, dim=0)
        y = torch.cat(labels_L, dim=0)    
        img_L, labels_L = [], []    
        with torch.no_grad():
            output = resnet50(img)
            p_idx = output.argmax(dim=1)
            gt_idx = y.argmax(dim=1)
            acc += (p_idx == gt_idx).sum().item()
            cnt += len(p_idx)
        if cnt > ms:
            print(acc / cnt)
            ms += 1000
print(acc / cnt)

  2%|▏         | 1042/50000 [00:10<09:47, 83.39it/s] 

0.78125


  4%|▍         | 2070/50000 [00:20<09:47, 81.56it/s] 

0.78662109375


  6%|▌         | 3091/50000 [00:31<09:30, 82.26it/s] 

...

0.7919921875


 16%|█▋        | 8212/50000 [01:26<08:01, 86.81it/s] 

0.79296875


 17%|█▋        | 8741/50000 [01:31<07:13, 95.23it/s] 


KeyboardInterrupt: 